In [ ]:
#pip install torch
#pip install torchvision
#pip install torchaudio

Step1: Import data from the google sheet

1. pip install google-api-python-client 
2. pip install google-auth-httplib2 
3. pip install google-auth-oauthlib 
4. pip install gspread

In [26]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd

In [27]:
#Defining score - in this case this is a google sheet
scopes  = ["https://www.googleapis.com/auth/spreadsheets"]

# Credentials to access the gsheet, we have define scope in scopes
creds = Credentials.from_service_account_file("credentials.json",scopes=scopes) 
client = gspread.authorize(creds)

In [28]:
# Open the workbook by its ID
sheet_id = "1hzPReAUH73ZtDqEbldRe-g4MyjfFH9ixoy8dfruPoXQ"
workbook = client.open_by_key(sheet_id)

In [35]:
sheet = workbook.worksheet('Meeting_notes')
values_list = sheet.get_all_values()
df = pd.DataFrame(values_list[1:], columns=values_list[0])  # Use the first row as column names

In [37]:
process_df = df[['Kitchen_name', 'House_name', 'Meeting_notes']]

Step 2: User Transformer model to summarize notes based on the first column

In [40]:
from transformers import pipeline

In [44]:
# Group by 'Kitchen_name' and join the meeting notes
grouped_notes = process_df.groupby('Kitchen_name')['Meeting_notes'].apply(lambda x: ' '.join(x)).reset_index()

In [ ]:
# Load BART model for summarization
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [71]:
def summarize_meeting_notes(notes):
    # Check if notes are empty
    if not notes.strip():
        return "No notes available."
    
    # Chunking if the text is too long
    max_input_length = 1024  # Adjust based on model requirements
    summaries = []
    
    # Split into chunks based on the maximum input length
    for i in range(0, len(notes), max_input_length):
        chunk = notes[i:i + max_input_length]
        
        try:
            # Summarize each chunk
            summary = summarizer(chunk, max_length=50, min_length=10, do_sample=False)
            summaries.append(summary[0]['summary_text'])
        except Exception as e:
            summaries.append(f"Error summarizing: {str(e)}")

    # Join all summaries into a single text for further summarization
    combined_summary = ' '.join(summaries)
    
    # Now create chunks for the final summarization if the combined summary is still too long
    final_summaries = []
    
    for i in range(0, len(combined_summary), max_input_length):
        final_chunk = combined_summary[i:i + max_input_length]
        
        try:
            final_summary = summarizer(final_chunk, max_length=50, min_length=10, do_sample=False)
            final_summaries.append(final_summary[0]['summary_text'])
        except Exception as e:
            final_summaries.append(f"Error summarizing final chunk: {str(e)}")

    # Return the final condensed summary
    return ' '.join(final_summaries)


In [72]:
# Apply summarization to the grouped meeting notes
grouped_notes['Summarized_Notes'] = grouped_notes['Meeting_notes'].apply(summarize_meeting_notes)

Your max_length is set to 50, but your input_length is only 23. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Your max_length is set to 50, but your input_length is only 47. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)
Your max_length is set to 50, but your input_length is only 29. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


In [73]:
# Display the result
print(grouped_notes[['Kitchen_name', 'Summarized_Notes']])

                        Kitchen_name  \
0                Coimbatore Kitchenn   
1                     Indore Kitchen   
2  Kitchen - Greater Noida - Chicago   
3                   SLC BLR RR Nagar   
4                 SLC HYD Kukatpally   
5                     SLC MAA Guindy   
6                 SLC PUN Dhankawadi   
7                SLC UDI Anant Nagar   
8                     Zagreb Kitchen   

                                    Summarized_Notes  
0  The Food Service Operations Review focused on ...  
1  The Food Feedback Meeting at Easton House focu...  
2  Food Service Feedback and Planning Meeting at ...  
3  The Food Feedback session at Salzburg House fo...  
4  The Food Service Feedback Meeting at Dijon Hou...  
5  Food Feedback meeting held at Essen House, Che...  
6  Residents expressed a desire for a more varied...  
7  Problem-solving and feedback session for Kilma...  
8  The Food Service Feedback and Improvement Meet...  


In [74]:
# Access the 'Kitchen_summary' tab
summary_sheet = workbook.worksheet('Kitchen_summary')

In [75]:
# Prepare data to append
data_to_append = grouped_notes[['Kitchen_name', 'Summarized_Notes']].values.tolist()

In [76]:
# Update the sheet with new data, starting from cell A1
summary_sheet.update('A1', [['Kitchen_name', 'Summarized_Notes']] + data_to_append)

print("Data successfully exported to Google Sheets!")

C:\Users\Neeraj.Mehta\AppData\Local\Temp\ipykernel_23068\4028896351.py:2: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  summary_sheet.update('A1', [['Kitchen_name', 'Summarized_Notes']] + data_to_append)


Data successfully exported to Google Sheets!
